In [1]:
# 1. 환경 설정 및 필수 라이브러리 임포트
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# newly imported libraries
from langchain_core.load import dumps, loads
from operator import itemgetter

# API 키 로드 (.env 파일에 설정되어 있어야 합니다)
load_dotenv()
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_paths = [
    "C:/Users/user/Downloads/삼성_2025Q4_conference_eng_presentation.pdf",
    "C:/Users/user/Downloads/삼성_2025Q4_script_eng_AudioScript.pdf"
]

docs = []

for path in file_paths:
    # 각 파일을 로드
    loader = PyPDFLoader(path)
    
    # load를 통해 문서의 각 페이지를 Document 객체로 변환
    docs.extend(loader.load())

print(f"총 로드된 페이지 수: {len(docs)}")
# AudioSciprt의 페이지 수: 34
# presentation의 페이지 수: 15
# 총합 로드 페이지 수: 49

총 로드된 페이지 수: 49


In [3]:
# 텍스트 1000자 단위로 자르고, 문맥 유지를 위해 50자씩 곂치도록 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=120
)

# 로드된 docs (presentation + audio transcript) 분할
splits = text_splitter.split_documents(docs)

print(f"분할된 청크 수: {len(splits)}\n")

# 삼성 presentation의 첫번째 슬라이드 text
print(f"첫번째 청크 예시:\n{splits[0].page_content}\n")

# 삼성 presentation의 두번째 슬라이드 text
print(f"두번쨰 청크 예시:\n{splits[1].page_content}")



분할된 청크 수: 96

첫번째 청크 예시:
SAMSUNG 
ELECTRONICS
Earnings Presentation: 
4Q 2025 Financial Results

두번쨰 청크 예시:
The financial information in this document are consolidated earnings results based on K-IFRS.
This document is provided for the convenience of investors only before the external audit on our 4Q 2025 financial results iscompleted. 
The Audit outcomes may cause some parts of this document to change. 
This document contains "forward-looking statements" - that is statements related to future not past events. 
In this context "forward-looking statements" often address our expected future business and financial performance 
and often contain words such as "expects" "anticipates" "intends" "plans" "believes" "seeks" or "will". 
"Forward-looking statements" by their nature address matters that are to different degrees uncertain. 
For us particular uncertainties which could adversely or positively affect our future results include:
·  The behavior of financial markets including fluctuatio

In [4]:
# GPU 사용 가능 여부 확인
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)

2.12.0.dev20260218+cu128
True
NVIDIA GeForce RTX 5060 Ti
12.8


In [5]:
# 허깅페이스 오픈소스 무료 임베딩 모델 활용 + GPU 활용
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': 'cuda'}
)

# 분할된 텍스트를 벡터 저장소에 인덱싱하여 저장
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# Vectorstore에 있는 저장된 벡터를 검색기로 변환하여 출력준비
retriever = vectorstore.as_retriever()

In [6]:
repo_id = "google/gemma-2-9b-it"

llm_endpoint = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_new_tokens=2048,
    temperature=0.1,
    huggingfacehub_api_token=hf_token,
)
chat_llm = ChatHuggingFace(llm=llm_endpoint)

## 6. Multi-Query RAG 구성

---여기서부터 기존 RAG와 달라짐---

### Step 1: Multi-Query 생성 체인
사용자 질문 1개 → LLM이 5가지 다른 표현으로 재작성

In [7]:
# Multi Query Prompt: 동일한 질문을 5가지 다른 관점으로 재작성
QUERY_PROMPT = ChatPromptTemplate.from_template("""
You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. 

Original question: {question}
""")

# query_chain: 질문 → 5개의 재작성된 질문 리스트
# StrOutputParser()로 string 변환 후 줄바꿈 기준으로 split
query_chain = (
    QUERY_PROMPT
    | chat_llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

print("Multi-Query 체인 준비 완료")

Multi-Query 체인 준비 완료


### Step 2 & 3: Retrieval + Unique Union

- `retriever.map()`: 5개의 질문 각각에 대해 retriever를 병렬 실행
- `get_unique_union()`: 5개의 결과 리스트를 합치고 중복 제거

In [8]:
def get_unique_union(documents: list[list]):
    """
    5개의 쿼리로 검색된 문서 리스트를 받아 중복을 제거하고 반환.
    
    Args:
        documents: [[doc1, doc2, ...], [doc3, doc4, ...], ...] 형태의 리스트
    Returns:
        중복이 제거된 Document 객체 리스트
    """
    # 중첩 리스트를 1차원으로 flatten하고, Document 객체를 JSON string으로 변환
    # (set() 비교를 위해 hashable한 string으로 변환 필요)
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    
    # set()으로 완전히 동일한 문서 string 제거
    unique_docs = list(set(flattened_docs))
    
    # 다시 Document 객체로 복원
    return [loads(doc) for doc in unique_docs]

In [ ]:
# retrieval_chain: 질문 → 5개 재작성 → 병렬 검색 → 중복 제거
#
# [수정] 기존 코드: QUERY_PROMPT | retriever.map() → TypeError 발생
# QUERY_PROMPT만 쓰면 ChatPromptValue 객체가 그대로 retriever에 전달됨
# retriever.map()은 string 리스트를 기대하므로 반드시 query_chain을 먼저 거쳐야 함
retrieval_chain = query_chain | retriever.map() | get_unique_union

# 테스트 실행
question = "삼성전자 2025 4분기 실적 발표에 대해서 알려줘"
docs = retrieval_chain.invoke({"question": question})

print(f"검색된 unique 문서 수: {len(docs)}개")
# Langsmith Trace
# https://smith.langchain.com/public/8478cf93-20e6-4876-8945-a5eba38b1f9c/r

검색된 unique 문서 수: 18개


C:\Users\user\AppData\Local\Temp\ipykernel_24072\3787532524.py:18: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


## 7. 최종 RAG 체인 구성

retrieval_chain으로 가져온 unique 문서들을 context로 사용해 최종 답변 생성

In [10]:
# 최종 답변 생성용 프롬프트
RAG_PROMPT = ChatPromptTemplate.from_template("""
당신은 삼성전자 실적발표 전문 AI 어시스턴트입니다.
제공된 컨텍스트를 바탕으로 질문에 상세히 답변하세요.
수치(숫자)를 정확히 포함하여 디테일하게 설명하세요.

#Context:
{context}

#Question:
{question}

#Answer:
""")

# context: retrieval_chain의 결과 (unique 문서들을 하나의 string으로 합침)
# question: 원본 사용자 질문을 itemgetter로 그대로 전달
final_rag_chain = (
    {
        "context": retrieval_chain,
        "question": itemgetter("question")
    }
    | RAG_PROMPT
    | chat_llm
    | StrOutputParser()
)

print("최종 RAG 체인 준비 완료")

최종 RAG 체인 준비 완료


## 8. 실행 및 테스트

In [ ]:
from langchain_teddynote.messages import stream_response

question = "삼성전자 2025 4분기 실적 발표에 대해서 알려줘"
answer = final_rag_chain.invoke({"question": question})

stream_response(answer)
# Langsmith Trace
# https://smith.langchain.com/public/5ea28f49-95a5-45e4-b7d8-208ec5eb900b/r

삼성전자는 2025년 4분기에 괄목할 만한 성과를 거두었습니다. 

**주요 실적:**

* **매출:** 93.8조원 (전년 동기 대비 11% 증가)
* **영업이익:** 20.1조원 (전년 동기 대비 33% 증가)
* **전년도 매출:** 333.6조원
* **전년도 영업이익:** 43.6조원

**주요 사업별 성과:**

* **디바이스솔루션(DS):** 
    * DRAM, NAND 비트 성장 및 ASP 상승으로 인해 4분기 매출이 44조원으로 전년 동기 대비 33% 증가했습니다.
    * 특히, HBM4 및 GDDR7 등 차세대 제품 출시로 시장 점유율 확대에 성공했습니다.
* **디지털익스피리언스(DX):** 
    * MX(스마트폰) 및 가전 사업의 성장이 둔화되었지만, 
    *  전반적으로 1분기 대비 매출이 증가했습니다.

**기타:**

* **환율 변동:** 달러 강세로 인해 1.6조원의 추가 영업이익이 발생했습니다.
* **CAPEX:** 지속적인 투자를 통해 미래 성장을 위한 기반을 다질 계획입니다.
* **지속가능성:** 에너지 절약 및 자원 순환을 위한 노력을 강화하고 있습니다.

**2026년 전망:**

* **전반적인 경기 불확실성:** 
    * 글로벌 경제 불황과 반도체 시장의 변동성 등으로 인해 2026년 상황은 불확실합니다.
* **핵심 전략:** 
    * 차세대 기술 개발 및 투자를 통해 경쟁력을 강화할 계획입니다.
    * 지속가능한 성장을 위한 노력을 지속적으로 추진할 것입니다.


